In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from feature_engine.encoding import OneHotEncoder

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

In [2]:
df = pd.read_csv('../datasets/raw/hour.csv')
df.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


## 1- Data Understanding
- What are the columns in the data? Explain each column.
- What columns contain missing values? And how do you intend to deal with them?
- Are there any columns that need conversion or re-encoding?
- Will the target column be CNT, registered, casual, and why did you choose this?

### Dataset characteristics

Both hour.csv and day.csv have the following fields, except hr which is not available in day.csv
	
	- instant: record index
	- dteday : date
	- season : season (1:springer, 2:summer, 3:fall, 4:winter)
	- yr : year (0: 2011, 1:2012)
	- mnth : month ( 1 to 12)
	- hr : hour (0 to 23)
	- holiday : weather day is holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)
	- weekday : day of the week
	- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
	+ weathersit : 
		- 1: Clear, Few clouds, Partly cloudy, Partly cloudy
		- 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
		- 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
		- 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
	- temp : Normalized temperature in Celsius. The values are divided to 41 (max)
	- atemp: Normalized feeling temperature in Celsius. The values are divided to 50 (max)
	- hum: Normalized humidity. The values are divided to 100 (max)
	- windspeed: Normalized wind speed. The values are divided to 67 (max)
	- casual: count of casual users
	- registered: count of registered users
	- cnt: count of total rental bikes including both casual and registered

In [ ]:
# Get columns with NA values
na_cols = [col for col in df.columns if df[col].isna().sum() > 0 ]
print(f"Columns with Null values: {na_cols}")


# Is there any columns that need any transformation or encoding?
SEASON = {1:'spring', 2:'summer', 3:'fall', 4:'winter'} # Mapping this dict with dataframe for getting insights
WEATHERSIT = {1:'clear', 2:'mist', 3:'light snow', 4:'heavy rain'} # Mapping this dict with dataframe for getting insights
NORMALIZE = {'temp':41, 'atemp':50, 'hum':100, 'windspeed':67} # Undo Normalize for getting real values, Before normalize
WORKDAY_OR_HOLIDAY = {0:'holiday', 1:'workday'}

def real_values(df):
    df = df.copy()
    for col, max_val in NORMALIZE.items():
        df[f"real_{col}"] = df[col] * max_val
    return df

df = real_values(df=df)

# Converting dtype to datetime type
df.dteday = pd.to_datetime(df.dteday, format='%Y-%m-%d')

# Getting real values
df['real_season'] = df.season.map(SEASON)
df['real_weathersit'] = df.weathersit.map(WEATHERSIT)
df['weekday_name'] = df.dteday.dt.day_name()
df['month_name'] = df.dteday.dt.month_name().str.slice(start=0, stop=3)
df['workday_or_holiday'] = df.workingday.map(WORKDAY_OR_HOLIDAY)

# Which feature I will select to be the target feature?
# I Will select "cnt" as target

# 1- Exploratory Data Analysis
- What is the distribution of the number of rented bicycles throughout the day?
- Does the season, holiday, or weekday affect the number of bikes rented?
- Do temperature, humidity, and wind affect the number of bicycles rented?
- Is there a connection between the columns?
- Are there any outliers that might affect the analysis or modeling?

In [ ]:
df.head()

In [ ]:
ohe = OneHotEncoder(variables='real_season')
ohe.fit_transform(df).head(30)

In [ ]:
df.real_season.unique()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# What is the distribution of the number of rented bicycles throughout the day?
# 1) Distribution of rentals throughout the day
hourly = df.groupby('hr')['cnt'].median()
sns.lineplot(x=hourly.index, y=hourly.values, ax=axes[0, 0])
axes[0, 0].set_title("Average Number of Rentals Throughout the Day")
axes[0, 0].set_xlabel("Hour of Day")
axes[0, 0].set_ylabel("Median Rentals")
axes[0, 0].grid(True)

# Does the season, holiday, or weekday affect the number of bikes rented?
# 2) Rentals by Season
season_order = ['spring', 'summer', 'fall', 'winter']
seasonal = df.groupby('real_season')['cnt'].median()
sns.barplot(x=seasonal.index, y=seasonal.values, order=season_order, ax=axes[0, 1])
axes[0, 1].set_title("Average Rentals by Season")
axes[0, 1].set_xlabel("Season")
axes[0, 1].set_ylabel("Median Rentals")
axes[0, 1].grid(True)

# 3) Rentals on Holidays vs Non-Holidays
holiday = df.groupby('holiday')['cnt'].median()
sns.barplot(x=holiday.index, y=holiday.values, ax=axes[1, 0])
axes[1, 0].set_title("Median Rentals on Holidays")
axes[1, 0].set_xlabel("Holiday (0=No, 1=Yes)")
axes[1, 0].set_ylabel("Median Rentals")
axes[1, 0].grid(True)

# 4) Rentals by Weekday
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
weekday = df.groupby('weekday_name')['cnt'].median()
sns.barplot(x=weekday.index, y=weekday.values, order=day_order, ax=axes[1, 1])
axes[1, 1].set_title("Median Rentals by Weekday")
axes[1, 1].set_xlabel("Weekday")
axes[1, 1].set_ylabel("Median Rentals")
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
cat_cols = ['season', 'holiday', 'weekday', 'workingday', 'weathersit']
num_cols = ['mnth', 'hr', 'temp', 'atemp', 'hum', 'windspeed',
            'casual', 'registered', 'cnt']

num_cols2 = num_cols.copy()
num_cols2.remove('casual'); num_cols2.remove('registered')

fig, axes = plt.subplots(1, 3, figsize=(22, 6)
                         )
# Do temperature, humidity, and wind affect the number of bicycles rented?
# Correlation: temp, atemp, hum, windspeed vs cnt
affect_corr = df[['temp', 'atemp', 'hum', 'windspeed', 'cnt']].corr()
affect_mask = np.triu(np.ones_like(affect_corr, dtype=bool))
sns.heatmap(affect_corr, annot=True, mask=affect_mask, ax=axes[0])
axes[0].set_title('Correlation: temp / hum / wind vs cnt')

# Is there a connection between the columns
# Correlation: all numerical columns 
corr = df[num_cols2].corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, annot=True, mask=mask, ax=axes[1])
axes[1].set_title("Correlation between numeric features")

# Are there any outliers that might affect the analysis or modeling?
# Boxplot: Outliers
df[['real_temp', 'real_atemp', 'real_hum', 'real_windspeed']].plot(
    kind='box', ax=axes[2]
)
axes[2].set_title("Checking for outliers")

plt.tight_layout()
plt.show()


In [ ]:
# Transforming columns with outliers (log(1 + x))
# df['log_casual'] = np.log1p(df['casual'])      # log(1 + x)
df['log_registered'] = np.log1p(df['registered'])
df['log_real_windspeed'] = np.log1p(df['real_windspeed'])
df['log_casual'] = np.log1p(df['casual'])

df[['log_registered', 'log_real_windspeed']].plot(kind='box')
plt.show()


## 2- Time-Series / Temporal Analysis
- How does the number of bicycles change over the course of an hour, day, or month?
- Is there any clear trend or seasonality?
- Is there any difference between weekdays and weekends?
- Is data needs scaling or smoothing before modeling?
* Official holidays are (Saturday, Sunday)

In [ ]:
# How does the number of bicycles change over the course of an hour, day, or month?
fig, axes = plt.subplots(3, 2, figsize=(16, 9))
MEAN = 'mean'
MEDIAN = 'median'

# Rush Hours
rush_hours = df.groupby('hr', as_index=False).agg(mean_rented_bikes=('cnt', MEDIAN))
sns.barplot(data=rush_hours, x='hr', y='mean_rented_bikes', ax=axes[0, 0])
axes[0, 0].set_title('Median number of rented bikes per Hour')
axes[0, 0].set_ylabel("Average Bikes Number")
axes[0, 0].set_xlabel("Hour")

# Rush Days
rush_day = df.groupby('weekday_name', as_index=False).agg(mean_rented_bikes=('cnt', MEDIAN))
sns.barplot(data=rush_day, x='weekday_name', y='mean_rented_bikes', order=day_order, ax=axes[0, 1])
axes[0, 1].set_title('Median number of rented bikes per Day')
axes[0, 1].set_ylabel("Average Bikes Number")
axes[0, 1].set_xlabel("Day")

# Rush Months
month_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

rush_month = df.groupby('month_name', as_index=False).agg(mean_rented_bikes=('cnt', MEDIAN))
sns.barplot(data=rush_month, x='month_name', y='mean_rented_bikes', order=month_order, ax=axes[1, 0])
axes[1, 0].set_title('Median number of rented bikes per Month')
axes[1, 0].set_ylabel("Average Bikes Number")
axes[1, 0].set_xlabel("Month")


# Is there any clear trend or seasonality?
rush_season = df.groupby('real_season', as_index=False).agg(mean_rented_bikes=('cnt', MEDIAN))
sns.barplot(data=rush_season, x='real_season', y='mean_rented_bikes', order=season_order, ax=axes[1, 1])
axes[1, 1].set_title('Median number of rented bikes per Season')
axes[1, 1].set_ylabel("Average Bikes Number")
axes[1, 1].set_xlabel("Season")


# Is there any difference between weekdays and weekends?
# Mean rented bikes for working days vs non-working days
df.groupby('workday_or_holiday', as_index=False)['cnt'].median() \
  .rename(columns={'workday_or_holiday': 'is_working_day', 'cnt': 'mean_rented_bikes'}).plot(kind='bar', x='is_working_day', y='mean_rented_bikes', ax=axes[2, 0], legend=False)
axes[2, 0].set_title("Defference between rented bikes in Weekdays VS (Weekends & Holidays)")
axes[2, 0].set_xlabel('')
axes[2, 0].set_ylabel('')
axes[2, 0].tick_params(axis='x', rotation=0)


# Hide last empty subplot
axes[2, 1].axis('off')

plt.tight_layout()
plt.show()

# Is data needs scaling or smoothing before modeling?
# Not realy because it normalized

## 3- Advanced Insights
- Is `Casual`, `Registered` Users have different behavior?
- What factors affect on `casual` vs `registered` Users?
- What is the best feature predictor number of bicycles to rent?
- Can we create a new feature to help the model? (is_weekend, rush_hour)

In [ ]:
def melting_casual_registered(col:str): # Using (mean)
    rush_feature = df.groupby(col)[['casual', 'registered']].mean().reset_index()

    # melt to make hue work
    rush_melt = rush_feature.melt(id_vars=col,
                                    value_vars=['casual', 'registered'],
                                    var_name='user_type',
                                    value_name='median_rented_bikes')
    return rush_melt


In [ ]:
# Is `Casual`, `Registered` Users have different behavior?

fig, axes = plt.subplots(3, 2, figsize=(20, 10))

# Function already defined by you:
# melting_casual_registered(col)

# Hour 
rush_hours_melt = melting_casual_registered('hr')
sns.barplot(
    data=rush_hours_melt,
    x='hr',
    y='median_rented_bikes',
    hue='user_type',
    ax=axes[0, 0]
)
axes[0, 0].set_title('Casual vs Registered — Median bikes per Hour')
axes[0, 0].set_xlabel('Hour')
axes[0, 0].set_ylabel('Median Bikes')

# Weekday Name
rush_weekdayname_melt = melting_casual_registered('weekday_name')
sns.barplot(
    data=rush_weekdayname_melt,
    x='weekday_name',
    y='median_rented_bikes',
    hue='user_type',
    order=day_order,
    ax=axes[0, 1]
)
axes[0, 1].set_title('Casual vs Registered — Median per Day')
axes[0, 1].set_xlabel('Day')
axes[0, 1].set_ylabel('Median Bikes')

# Month Name 
rush_month_melt = melting_casual_registered('month_name')
sns.barplot(
    data=rush_month_melt,
    x='month_name',
    y='median_rented_bikes',
    hue='user_type',
    order=month_order,
    ax=axes[1, 0]
)
axes[1, 0].set_title('Casual vs Registered — Median per Month')
axes[1, 0].set_xlabel('Month')
axes[1, 0].set_ylabel('Median Bikes')

# Season 
rush_season_melt = melting_casual_registered('real_season')
sns.barplot(
    data=rush_season_melt,
    x='real_season',
    y='median_rented_bikes',
    hue='user_type',
    order=season_order,
    ax=axes[1, 1]
)
axes[1, 1].set_title('Casual vs Registered — Median per Season')
axes[1, 1].set_xlabel('Season')
axes[1, 1].set_ylabel('Median Bikes')

# Working Day vs Holiday ---
rush_workday_melt = melting_casual_registered('workday_or_holiday')
sns.barplot(
    data=rush_workday_melt,
    x='workday_or_holiday',
    y='median_rented_bikes',
    hue='user_type',
    ax=axes[2, 0]
)
axes[2, 0].set_title("Casual vs Registered — Workingday vs (Holiday & Weekend)")
axes[2, 0].set_xlabel("Workday (1) | Weekend/Holiday (0)")
axes[2, 0].set_ylabel("Median Bikes")

# Hide last empty subplot
axes[2, 1].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# What factors affect on `casual` vs `registered` Users?
casual_users = num_cols.copy()
casual_users.remove('cnt'); casual_users.remove('registered')

registered_users = num_cols.copy()
registered_users.remove('cnt'); registered_users.remove('casual')

fig, axes = plt.subplots(1, 2, figsize=(22, 6))

# Correlation for Casual Users
affect_casual_corr = df[casual_users].corr()
affect_casual_mask = np.triu(np.ones_like(affect_casual_corr, dtype=bool))
sns.heatmap(affect_casual_corr, annot=True, mask=affect_casual_mask, ax=axes[0])
axes[0].set_title('Correlation between features affecting Casual Users')

# Correlation for Registered Users
affect_registered_corr = df[registered_users].corr()
affect_registered_mask = np.triu(np.ones_like(affect_registered_corr, dtype=bool))
sns.heatmap(affect_registered_corr, annot=True, mask=affect_registered_mask, ax=axes[1])
axes[1].set_title('Correlation between features affecting Registered Users')

plt.tight_layout()
plt.show()

# We can see that [hr, temp, atemp] have a positive relation with (casual, registered) users
# And [hum] have a negative relation with (casual, registered) users

# Can we create a new feature to help the model? (is_weekend, rush_hour)
# there is two columns (holiday & workingday) that can replace is_weekend
# So I can create a column that tells me if it is a rush hour or not for either (casual or registered) Users

# IS Rush hours for Registered Users

registered_rush_hrs = df.groupby('hr', as_index=False)['registered'].sum().sort_values('registered',ascending=False)['hr'].head().tolist()
df['registered_rush_hrs'] = df['hr'].isin(registered_rush_hrs).astype(int)


# IS Rush hours for Casual Users
casual_rush_hrs = df.groupby('hr', as_index=False)['casual'].sum().sort_values('casual',ascending=False)['hr'].head().tolist()
df['casual_rush_hrs'] = df['hr'].isin(casual_rush_hrs).astype(int)

# IS Rush hours for all users
cnt_rush_hrs = df.groupby('hr', as_index=False)['cnt'].sum().sort_values('cnt',ascending=False)['hr'].head().tolist()
df['cnt_rush_hrs'] = df['hr'].isin(registered_rush_hrs).astype(int)


In [ ]:
# What factors affect on `casual` vs `registered` Users?
casual_users = num_cols.copy()
casual_users.remove('cnt'); casual_users.remove('registered'); casual_users.append('casual_rush_hrs')

registered_users = num_cols.copy()
registered_users.remove('cnt'); registered_users.remove('casual'); registered_users.append('registered_rush_hrs')


fig, axes = plt.subplots(1, 3, figsize=(22, 6))

# Correlation for Casual Users
affect_casual_corr = df[casual_users].corr()
affect_casual_mask = np.triu(np.ones_like(affect_casual_corr, dtype=bool))
sns.heatmap(affect_casual_corr, annot=True, mask=affect_casual_mask, ax=axes[0])
axes[0].set_title('Correlation between features affecting Casual Users')

# Correlation for Registered Users
affect_registered_corr = df[registered_users].corr()
affect_registered_mask = np.triu(np.ones_like(affect_registered_corr, dtype=bool))
sns.heatmap(affect_registered_corr, annot=True, mask=affect_registered_mask, ax=axes[1])
axes[1].set_title('Correlation between features affecting Registered Users')

# Correlation for all Users
num_cols.append('cnt_rush_hrs')
all_users_num_cols = num_cols.copy(); all_users_num_cols.remove('casual'); all_users_num_cols.remove('registered')
affect_user_corr = df[all_users_num_cols].corr()
affect_user_mask = np.triu(np.ones_like(affect_user_corr, dtype=bool))
sns.heatmap(affect_user_corr, annot=True, mask=affect_user_mask, ax=axes[2])
axes[2].set_title('Correlation between features affecting All Users')
axes[2].tick_params(axis='y', rotation=0)

plt.tight_layout()
plt.show()

## 4- Understanding the relationship between categorical columns
- Using Chi-Square test to know if there is a relationship or not
- Then Using Cramers_v to determine the strength of the relationship between the category columns

In [ ]:
from scipy.stats import chi2_contingency

cat_cols = ['season', 'holiday', 'weekday', 'workingday', 'weathersit']

chi2_pvals = pd.DataFrame(index=cat_cols, columns=cat_cols)

for col1 in cat_cols:
    for col2 in cat_cols:
        if col1 == col2:
            chi2_pvals.loc[col1, col2] = 1.0 
        else:
            contingency = pd.crosstab(df[col1], df[col2])
            chi2, p, dof, expected = chi2_contingency(contingency)
            chi2_pvals.loc[col1, col2] = p

chi2_pvals = chi2_pvals.astype(float)

print("Chi-square p-values matrix for categorical columns:")
display(chi2_pvals)


In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(
    chi2_pvals,
    annot=True,
    fmt=".4f",
    # cmap="coolwarm_r",  
    cbar_kws={'label': 'p-value'}
)
plt.title("Chi-square P-values Heatmap for Categorical Columns")
plt.show()

In [ ]:
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]  
    n = confusion_matrix.sum().sum()              
    r, k = confusion_matrix.shape                 
    return np.sqrt(chi2 / (n * (min(r, k) - 1)))

cat_cols = ['season', 'holiday', 'weekday', 'workingday', 'weathersit']

for i, col1 in enumerate(cat_cols):
    for col2 in cat_cols[i+1:]:
        v = cramers_v(df[col1], df[col2])
        print(f"Cramér's V between {col1} and {col2}: {v:.3f}")


In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

df_daily = df.groupby('dteday')['cnt'].sum()

result = seasonal_decompose(df_daily, model='additive', period=30)
result.plot()
plt.show()


## 5- Modeling
### `Note: That you have to split data to train and test dataframes, Before Selecting features for your model to not cause a data leakage`
- What type of model do you think is suitable for predicting the number of bicycles? [Time Series, Regression, ML model]
- What matrix will you use to measure the model's performance? [R2_score, MAE, RMSE]
- If there is overfitting, how will you fix it?
- Do you need to do cross-validation? If so, how?

    * Regression model
    * feature Engineering
    * Feature Importance
    * Time series Trend / Seasonal Decomposition

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from feature_engine.creation import CyclicalFeatures
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
import optuna
from optuna.integration import OptunaSearchCV
from sklearn.metrics import r2_score, root_mean_squared_error, mean_absolute_error

import warnings
warnings.filterwarnings("ignore")


cols = ['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday',
       'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
       'cnt']

X = df[cols].drop(labels='cnt', axis=1)
y = df['cnt']

class RushHourTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, variables: list[str], target: str = "cnt", top_n: int = 5):
        self.variables = variables
        self.target = target
        self.top_n = top_n
        self.top_hours_ = {}

    def fit(self, X: pd.DataFrame, y: pd.Series = None):
        if y is None:
            raise ValueError("y must be provided for RushHourTransformer fit")

        for var in self.variables:
            temp_df = pd.DataFrame({var: X[var], self.target: y})
            top_hours = (
                temp_df.groupby(var)[self.target]
                .sum()
                .sort_values(ascending=False)
                .head(self.top_n)
                .index.tolist()
            )
            self.top_hours_[var] = top_hours

        return self

    def transform(self, X: pd.DataFrame, y=None):
        X_transformed = X.copy()
        for var in self.variables:
            X_transformed[f"{var}_rush_hr"] = (
                X_transformed[var].isin(self.top_hours_[var]).astype(int)
            )
        return X_transformed


# Not Shuffling beacause it's a datetime data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, shuffle=False, random_state=42)

# Splitting training columns to it's type
cyclical_cols = ['season', 'mnth', 'weekday', 'hr']
binary_cols = ['yr', 'holiday', 'workingday']
new_feat = [] # 'casual', 'registered'
num_cols = ['temp', 'atemp', 'hum', 'windspeed']
cat_cols = ['weathersit']

# Making pipe lines for each type of columns 
cyclical_pipe = Pipeline(steps=[
    ('cyclical_transformation', CyclicalFeatures(drop_original=True))
])

numeric_pipe = Pipeline(steps=[
    ('scalling_numbers', StandardScaler())
])

categories_pipe = Pipeline(steps=[
    ('ohe', OneHotEncoder(sparse_output=False))
])

# Specifing list of columns for each transfromer
ct = ColumnTransformer(transformers=[
    ('cyclical', cyclical_pipe, cyclical_cols),
    ('scalling', numeric_pipe, num_cols),
    ('ohe', categories_pipe, cat_cols)
    ],
    remainder='drop').set_output(transform='pandas')

# Making a functiontransformer to get rush hours for (casual, registered)
# rush_transformer = FunctionTransformer(RushHourTransformer(variables=['hr']), validate=False)
    
def compare_between_models(models: dict):
    """Fucntion to make a comparing between ML models"""

    results = {}

    for model_name, model_obj in models.items():

        full_pipeline = Pipeline(steps=[
            ('rush_hrs', RushHourTransformer(variables=["hr"], target="cnt", top_n=5)),
            ('preprocessing', ct),
            ('model', model_obj)
        ])

        # Trian
        full_pipeline.fit(X_train, y_train)

        # Predict
        y_pred_train = full_pipeline.predict(X_train)
        y_pred_test = full_pipeline.predict(X_test)

        # Store metrics
        results[model_name] = {
            "r2_train": r2_score(y_train, y_pred_train),
            "r2_test": r2_score(y_test, y_pred_test),
            "rmse_train": root_mean_squared_error(y_train, y_pred_train),
            "rmse_test": root_mean_squared_error(y_test, y_pred_test),
            "mae_train": mean_absolute_error(y_train, y_pred_train),
            "mae_test": mean_absolute_error(y_test, y_pred_test)
        }

    return results

models = {'XGBRegressor': XGBRegressor(),
          'CatBoostRegressor': CatBoostRegressor(verbose=0),
          'LGBMRegressor': LGBMRegressor()}

models_comparing_result = pd.DataFrame(compare_between_models(models=models))


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000135 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 323
[LightGBM] [Info] Number of data points in the train set: 13903, number of used features: 15
[LightGBM] [Info] Start training from score 174.639143


In [6]:
models_comparing_result

,XGBRegressor,CatBoostRegressor,LGBMRegressor
r2_train,0.946035,0.925754,0.903382
r2_test,0.673023,0.679482,0.670417
rmse_train,38.783062,45.490505,51.893581
rmse_test,126.077324,124.825944,126.578796
mae_train,25.809240,30.426882,34.918496
mae_test,89.116280,89.290588,90.154502


In [ ]:
# Hyperparamter Optimization using GridSearchCV

# Base CatBoost model
cat_model = CatBoostRegressor(
    verbose=0,
    random_state=42
)

# Full pipeline
full_pipeline = Pipeline(steps=[
    ('rush_hrs', rush_transformer),
    ('preprocessing', ct),
    ('model', cat_model)
])

# Hyperparameter grid
param_grid = {
    'model__iterations': [500, 1000],       
    'model__depth': [4, 6, 8],              
    'model__learning_rate': [0.01, 0.05, 0.1],
    'model__l2_leaf_reg': [1, 3, 5]         
}

# GridSearchCV 
grid_search = GridSearchCV(
    estimator=full_pipeline,
    param_grid=param_grid,
    cv=3,                    
    scoring='r2',      
    n_jobs=-1,
    verbose=0
)

# Train 
print("GridSearchCV Searching for Best HyperParameters ...")
grid_search.fit(X_train, y_train)

# Best Modle
best_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)

# train & test
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

results = {
    "r2_train": r2_score(y_train, y_pred_train),
    "r2_test": r2_score(y_test, y_pred_test),
    "rmse_train": root_mean_squared_error(y_train, y_pred_train),
    "rmse_test": root_mean_squared_error(y_test, y_pred_test),
    "mae_train": mean_absolute_error(y_train, y_pred_train),
    "mae_test": mean_absolute_error(y_test, y_pred_test)
}

print(results)


In [ ]:
# Hyperparamter Optimization using Optuna

optuna.logging.set_verbosity(optuna.logging.ERROR)

# Base CatBoost model 
cat_model = CatBoostRegressor(
    verbose=0,
    random_state=42,
    early_stopping_rounds=50
)

# Full pipeline 
full_pipeline = Pipeline(steps=[
    ('rush_hrs', rush_transformer),
    ('preprocessing', ct),
    ('model', cat_model)
])

# Define parameter search space 
param_distributions = {
    'model__iterations': optuna.distributions.IntDistribution(500, 1500),
    'model__depth': optuna.distributions.IntDistribution(4, 10),
    'model__learning_rate': optuna.distributions.FloatDistribution(0.01, 0.2, log=True),
    'model__l2_leaf_reg': optuna.distributions.FloatDistribution(1, 10)
}

# Optuna Search 
optuna_search = OptunaSearchCV(
    estimator=full_pipeline,
    param_distributions=param_distributions,
    cv=3,
    scoring='r2',
    n_trials=30, 
    n_jobs=-1,
    verbose=0,
    random_state=42
)

# Train 
print("Optuna Searching for Best HyperParamters ...")
optuna_search.fit(X_train, y_train)

# Best Model
best_model = optuna_search.best_estimator_
print("Best Parameters:", optuna_search.best_params_)

# train & test 
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

results = {
    "r2_train": r2_score(y_train, y_pred_train),
    "r2_test": r2_score(y_test, y_pred_test),
    "rmse_train": root_mean_squared_error(y_train, y_pred_train),
    "rmse_test": root_mean_squared_error(y_test, y_pred_test),
    "mae_train": mean_absolute_error(y_train, y_pred_train),
    "mae_test": mean_absolute_error(y_test, y_pred_test)
}

print(results)


In [ ]:
comparing_df = pd.DataFrame(models_comparing_result)
comparing_df

In [ ]:
comparing_df_t = comparing_df.T
models = comparing_df_t.index.tolist()

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# R2 Test
axes[0].bar(models, comparing_df_t["r2_test"])
axes[0].set_title("R2 (Test)")
axes[0].set_ylabel("R2 (higher is better)")
axes[0].set_ylim(0, 1.05)
axes[0].grid(axis='y', linestyle='--', alpha=0.3)
axes[0].tick_params(axis='x', rotation=15)

# RMSE Test
axes[1].bar(models, comparing_df_t["rmse_test"])
axes[1].set_title("RMSE (Test)")
axes[1].set_ylabel("RMSE (lower is better)")
axes[1].grid(axis='y', linestyle='--', alpha=0.3)
axes[1].tick_params(axis='x', rotation=15)

# MAE Test
axes[2].bar(models, comparing_df_t["mae_test"])
axes[2].set_title("MAE (Test)")
axes[2].set_ylabel("MAE (lower is better)")
axes[2].grid(axis='y', linestyle='--', alpha=0.3)
axes[2].tick_params(axis='x', rotation=15)

plt.tight_layout()
plt.show()


So We can see that
* CatBoostRegressor is the best base model 
* Next step is to make a Hyperparameter optimization for it
* Then Saving model and It's Pipeline

## 6- Recommendation / Insights
- What are the most important factors that influence bicycle usage?
- If we were a bike-sharing company, what steps could we take based on this analysis?
- What is the best time to increase the number of bicycles at stations?